In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
# 1. Charger les données
iris = load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names
features_names = iris.feature_names
dataset = pd.DataFrame(X, columns=features_names)
dataset['target'] = y

# 2. Afficher
dataset.head()

In [ ]:
# 3. Scatter plot des deux premières dimensions
plt.figure(figsize=(8, 6))
for label, color, marker in zip(list(dataset['target'].unique()), ['blue', 'green', 'red'], ['o', 's', 'x']):
    idx = np.where(y == label)
    plt.scatter(X[idx, 0], X[idx, 2], c=color, marker=marker, label=iris.target_names[label])
plt.xlabel(iris.feature_names[0])
plt.ylabel(iris.feature_names[2])
plt.title("Iris : Sepal Length vs Petal Length")
plt.legend()
plt.show()


In [ ]:
# 3. Scatter plot : Sepal Width vs Petal Width
plt.figure(figsize=(8, 6))
for label, color, marker in zip([0, 1, 2], ['blue', 'green', 'red'], ['o', 's', 'x']):
    idx = np.where(y == label)
    plt.scatter(X[idx, 1], X[idx, 3], c=color, marker=marker, label=iris.target_names[label])
plt.xlabel(iris.feature_names[1])  # Sepal Width
plt.ylabel(iris.feature_names[3])  # Petal Width
plt.title("Iris : Sepal Width vs Petal Width")
plt.legend()
plt.show()


In [ ]:
class_counts = pd.Series(y).value_counts(normalize=True)
print(f"Le pourcantage de chaque classe est : \n{np.round(class_counts*100,2)}")

In [ ]:
# 4. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
# Après division
print("\nProportions dans y_train :")
print(pd.Series(y_train).value_counts(normalize=True) * 100)

print("\nProportions dans y_test :")
print(pd.Series(y_test).value_counts(normalize=True) * 100)


In [ ]:
# 5. Entraîner le modèle multinomial
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
model.fit(X_train, y_train)

In [ ]:
# 6. Afficher les coefficients
coef_df = pd.DataFrame(model.coef_, columns=iris.feature_names)
coef_df['Classe'] = iris.target_names
coef_df

In [ ]:
# 7. Matrice de confusion
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("\nMatrice de confusion :\n", cm)

In [ ]:
# Tracer la matrice de confusion

fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(cm, cmap=plt.cm.Blues)

# Annotation de chaque cellule
n_classes = len(class_names)
for i in range(n_classes):
    for j in range(n_classes):
        ax.text(j, i, cm[i, j],
                ha="center", va="center",
                color="white" if cm[i, j] > cm.max()/2 else "black")

# Labels, titre, et ticks
ax.set_xticks(np.arange(n_classes))
ax.set_yticks(np.arange(n_classes))
ax.set_xticklabels(class_names, rotation=45, ha="center")
ax.set_yticklabels(class_names)
ax.set_xlabel('Prédictions')
ax.set_ylabel('Valeurs réelles')
ax.set_title('Matrice de Confusion — Iris')

# Barre de couleur
fig.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

In [ ]:

# 8. Rapport de classification
print("\nRapport de classification :\n",
      classification_report(y_test, y_pred, target_names=iris.target_names))

In [ ]:
# 9. Frontière de décision sur Petal Length & Petal Width
X2 = iris.data[:, [2, 3]]
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y, test_size=0.3, random_state=42, stratify=y
)
model2 = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
model2.fit(X2_train, y2_train)

In [ ]:
x_min, x_max = X2[:, 0].min() , X2[:, 0].max() 
y_min, y_max = X2[:, 1].min() , X2[:, 1].max() 
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.2),
                     np.arange(y_min, y_max, 0.2))
Z = model2.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

In [ ]:
plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, alpha=0.5, cmap='coolwarm')
plt.scatter(X2[:, 0], X2[:, 1], c=y, edgecolor='k', cmap='coolwarm')
plt.xlabel('Petal Length')
plt.ylabel('Petal Width')
plt.title('Frontière de Décision - Iris (Multiclass)')
plt.show()